In [1]:
import os, sys
from python_tools import *

python tools loaded.


In [2]:
import importlib
import pickle


class PickleProtocol:
    def __init__(self, level):
        self.previous = pickle.HIGHEST_PROTOCOL
        self.level = level

    def __enter__(self):
        importlib.reload(pickle)
        pickle.HIGHEST_PROTOCOL = self.level

    def __exit__(self, *exc):
        importlib.reload(pickle)
        pickle.HIGHEST_PROTOCOL = self.previous


def pickle_protocol(level):
    return PickleProtocol(level)

In [3]:
#function for mergning in event weights
def merge_weights(file,tree_names,var_suffix,df):
    for i in range(len(tree_names)):
        w_df = uproot.open(file+":"+tree_names[i]).arrays(library="pd")
        wght_name="wght_"+var_suffix[i]
        w_df = w_df.rename(columns={"wght": wght_name})
        df = df.merge(w_df[["iev",wght_name]],how="left",on=["iev"])
    return df

In [4]:
#function for combining all the event weights from the root files I've made...
def merge_weights_simple(filepath,knob_name,df,qual=""):
    filename="%s/%s_weights_%s.root"%(filepath,qual,knob_name)
    tree_names = [ "%s_twk%d"%(knob_name,i) for i in [0,1,3,4] ]
    var_suffixes = [ "%s_n2"%knob_name,"%s_n1"%knob_name,"%s_p1"%knob_name,"%s_p2"%knob_name ]
    return merge_weights(filename,tree_names,var_suffixes,df)

In [6]:
mass_dict = {}
#mass_dict

In [7]:
knob_names = ["FrCEx_N","FrAbs_N","FrInel_N","FrPiProd_N",#"MFP_N",
              "FrCEx_pi","FrAbs_pi","FrInel_pi","FrPiProd_pi"]#"MFP_pi",
              #"FormZone"]

In [17]:
for run in range(6,11):

    print("Running run %d ... "%run)
    
    rstring = "run_%d_gntp"%run
    gst_file = uproot.open("/Users/wketchum/Data/LDMX/eN_Ti_Base/%s.0.gst.root:gst"%rstring)
    gst_keys = gst_file.keys()
    
    df_gst_keys = gst_keys[0:74]+gst_keys[80:81]+gst_keys[88:]
    df_gst_ip_keys = gst_keys[0:1]+gst_keys[74:80]
    df_gst_fp_keys = gst_keys[0:1]+gst_keys[81:88]
    
    df_gst = gst_file.arrays(df_gst_keys,library="pd")
    df_gst_ip = gst_file.arrays(df_gst_ip_keys,library="pd")
    df_gst_fp = gst_file.arrays(df_gst_fp_keys,library="pd")
    
    for kn in knob_names:
        print("\tProcess knob %s"%kn)
        df_gst = merge_weights_simple(filepath="/Users/wketchum/Data/LDMX/eN_Ti_Rw_FSI/%s"%rstring,
                                      knob_name=kn,df=df_gst,qual=rstring)
    
    for pdg in set(df_gst_ip["pdgi"].values):
#        print(pdg)
        m=particle.Particle.from_pdgid(pdg).mass
        if not m: m=0.
        mass_dict[pdg] = m*1e-3
    for pdg in set(df_gst_fp["pdgf"].values):
        m=particle.Particle.from_pdgid(pdg).mass
        if not m: m=0.
        mass_dict[pdg] = m*1e-3
    
    df_gst_ip["mass"] = df_gst_ip["pdgi"].apply(lambda x: mass_dict[x])
    df_gst_fp["mass"] = df_gst_fp["pdgf"].apply(lambda x: mass_dict[x])
    
    df_gst_ip["kei"] = df_gst_ip["Ei"] - df_gst_ip["mass"]
    df_gst_fp["kef"] = df_gst_fp["Ef"] - df_gst_fp["mass"]
    
    df_gst["ptl"] = np.sqrt(df_gst["pxl"]*df_gst["pxl"]+df_gst["pyl"]*df_gst["pyl"])
  
    outfile="/Users/wketchum/Data/LDMX/eN_DataFrames_11Aug2021/%s_FSI_df.hdf"%rstring
    with pickle_protocol(4):
        df_gst.to_hdf(outfile,"gst_df",mode="w")
        df_gst_ip.to_hdf(outfile,"gst_ip_df",mode="a")
        df_gst_fp.to_hdf(outfile,"gst_fp_df",mode="a")
      

Running run 6 ... 
	Process knob FrCEx_N
	Process knob FrAbs_N
	Process knob FrInel_N
	Process knob FrPiProd_N
	Process knob FrCEx_pi
	Process knob FrAbs_pi
	Process knob FrInel_pi
	Process knob FrPiProd_pi
Running run 7 ... 
	Process knob FrCEx_N
	Process knob FrAbs_N
	Process knob FrInel_N
	Process knob FrPiProd_N
	Process knob FrCEx_pi
	Process knob FrAbs_pi
	Process knob FrInel_pi
	Process knob FrPiProd_pi
Running run 8 ... 
	Process knob FrCEx_N
	Process knob FrAbs_N
	Process knob FrInel_N
	Process knob FrPiProd_N
	Process knob FrCEx_pi
	Process knob FrAbs_pi
	Process knob FrInel_pi
	Process knob FrPiProd_pi
Running run 9 ... 
	Process knob FrCEx_N
	Process knob FrAbs_N
	Process knob FrInel_N
	Process knob FrPiProd_N
	Process knob FrCEx_pi
	Process knob FrAbs_pi
	Process knob FrInel_pi
	Process knob FrPiProd_pi
Running run 10 ... 
	Process knob FrCEx_N
	Process knob FrAbs_N
	Process knob FrInel_N
	Process knob FrPiProd_N
	Process knob FrCEx_pi
	Process knob FrAbs_pi
	Process knob 

In [ ]:
gst_file = uproot.open("/Users/wketchum/Data/LDMX/gntp.0.gst.root:gst")
gst_keys = gst_file.keys()

In [ ]:
df_gst_keys = gst_keys[0:74]+gst_keys[80:81]+gst_keys[88:]
df_gst_ip_keys = gst_keys[0:1]+gst_keys[74:80]
df_gst_fp_keys = gst_keys[0:1]+gst_keys[81:88]

In [ ]:
df_gst = gst_file.arrays(df_gst_keys,library="pd")

In [ ]:
df_gst_ip = gst_file.arrays(df_gst_ip_keys,library="pd")

In [ ]:
df_gst_fp = gst_file.arrays(df_gst_fp_keys,library="pd")

In [ ]:
knob_names

In [ ]:
for kn in knob_names:
    df_gst = merge_weights_simple(filepath="/Users/wketchum/Data/LDMX",knob_name=kn,df=df_gst,qual="FSIFix")

In [ ]:
df_gst

In [ ]:
df_gst_ip["mass"] = df_gst_ip["pdgi"].apply(lambda x: mass_dict[x])
df_gst_fp["mass"] = df_gst_fp["pdgf"].apply(lambda x: mass_dict[x])

In [ ]:
df_gst_ip["kei"] = df_gst_ip["Ei"] - df_gst_ip["mass"]
df_gst_fp["kef"] = df_gst_fp["Ef"] - df_gst_fp["mass"]


In [ ]:
df_gst_fp

In [ ]:
with pickle_protocol(4):
    df_gst.to_hdf("/Users/wketchum/Data/LDMX/base_generation_with_weights_13Jun_FSIFix.hdf","gst_df",mode="w")
    df_gst_ip.to_hdf("/Users/wketchum/Data/LDMX/base_generation_with_weights_13Jun_FSIFix.hdf","gst_ip_df",mode="a")
    df_gst_fp.to_hdf("/Users/wketchum/Data/LDMX/base_generation_with_weights_13Jun_FSIFix.hdf","gst_fp_df",mode="a")
    

In [ ]:
df_gst["ptl"] = np.sqrt(df_gst["pxl"]*df_gst["pxl"]+df_gst["pyl"]*df_gst["pyl"])

In [ ]:
df_gst[["pxl","pyl","pzl","ptl"]]

In [ ]:
df_gst_ptlcut_index = df_gst[df_gst["ptl"]>0.4].index

In [ ]:
df_gst_ptlcut_index

In [ ]:
df_gst_fp.join(df_gst["ptl"],on='entry').query("ptl>0.4")

In [ ]:
df_gst_fp.join(df_gst["ptl"],on='entry')

In [ ]:
df_gst_ip.join(df_gst["wght_FrCEx_pi_p1"],on='entry').query("wght_FrCEx_pi_p1!=1")

In [ ]:
df_gst["cthl"]